In [1]:
#packages
import pandas as pd
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
# loading data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
train_df.head()
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [3]:
#starting h2o
h2o.init(max_mem_size= '10G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_345"; OpenJDK Runtime Environment (Zulu 8.64.0.19-CA-macos-aarch64) (build 1.8.0_345-b01); OpenJDK 64-Bit Server VM (Zulu 8.64.0.19-CA-macos-aarch64) (build 25.345-b01, mixed mode)
  Starting server from /Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/yj/nrb4536x26s8lfxkz_6kzf100000gn/T/tmpex11et_x
  JVM stdout: /var/folders/yj/nrb4536x26s8lfxkz_6kzf100000gn/T/tmpex11et_x/h2o_hectorcozar_started_from_python.out
  JVM stderr: /var/folders/yj/nrb4536x26s8lfxkz_6kzf100000gn/T/tmpex11et_x/h2o_hectorcozar_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,"2 years, 3 months and 28 days !!!"
H2O_cluster_name:,H2O_from_python_hectorcozar_sb8f8o
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.89 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
# loading data as h2o object
df = h2o.import_file(path = '../data/train.csv')
df_sample = h2o.import_file(path = '../data/test.csv')


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
#describe dataset
df.describe(chunk_summary=True)

Rows:1460
Cols:81

Chunk compression summary: 


,chunk_type,chunk_name,count,count_percentage,size,size_percentage
0,C0L,Constant long,312,9.629629,24.4 KB,7.184719
1,CBS,Binary,113,3.487654,8.3 KB,2.439523
2,CXI,Sparse Integers,213,6.574074,17.4 KB,5.140988
3,C1,1-Byte Integers,51,1.574074,5.2 KB,1.534525
4,C1N,1-Byte Integers (w/o NAs),1864,57.530860,190.2 KB,56.068726
5,C1S,1-Byte Fractions,195,6.018518,22.9 KB,6.764171
6,C2,2-Byte Integers,436,13.456790,60.0 KB,17.694674
7,C2S,2-Byte Fractions,17,0.524691,2.6 KB,0.767406
8,C4,4-Byte Integers,39,1.203704,8.2 KB,2.405269



Frame distribution summary: 


,,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
0,127.0.0.1:54321,339.3 KB,1460.0,40.0,3240.0
1,mean,339.3 KB,1460.0,40.0,3240.0
2,min,339.3 KB,1460.0,40.0,3240.0
3,max,339.3 KB,1460.0,40.0,3240.0
4,stddev,0 B,0.0,0.0,0.0
5,total,339.3 KB,1460.0,40.0,3240.0


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
type,int,int,enum,int,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,enum,enum,enum,enum,enum,int,enum,enum,enum,enum,enum,enum,enum,int,enum,int,int,int,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int,int,enum,int,enum,int,enum,enum,int,enum,int,int,enum,enum,enum,int,int,int,int,int,int,enum,enum,enum,int,int,int,enum,enum,int
mins,1.0,20.0,,21.0,1300.0,,,,,,,,,,,,,1.0,1.0,1872.0,1950.0,,,,,,0.0,,,,,,,,0.0,,0.0,0.0,0.0,,,,,334.0,0.0,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,,2.0,,0.0,,,1900.0,,0.0,0.0,,,,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,1.0,2006.0,,,34900.0
mean,730.5,56.897260273972606,,70.04995836802665,10516.828082191782,,,,,,,,,,,,,6.0993150684931505,5.575342465753424,1971.267808219178,1984.8657534246574,,,,,,103.68526170798899,,,,,,,,443.6397260273973,,46.54931506849315,567.2404109589041,1057.4294520547944,,,,,1162.626712328767,346.99246575342465,5.844520547945206,1515.463698630137,0.42534246575342466,0.057534246575342465,1.5650684931506849,0.38287671232876713,2.8664383561643834,1.0465753424657538,,6.517808219178082,,0.613013698630137,,,1978.5061638868744,,1.7671232876712328,472.9801369863014,,,,94.2445205479452,46.66027397260274,21.954109589041096,3.4095890410958902,15.060958904109588,2.758904109589041,,,,43.489041095890414,6.321917808219178,2007.8157534246575,,,180921.19589041095
maxs,1460.0,190.0,,313.0,215245.0,,,,,,,,,,,,,10.0,9.0,2010.0,2010.0,,,,,,1600.0,,,,,,,,5644.0,,1474.0,2336.0,6110.0,,,,,4692.0,2065.0,572.0,5642.0,3.0,2.0,3.0,2.0,8.0,3.0,,14.0,,3.0,,,2010.0,,4.0,1418.0,,,,857.0,547.0,552.0,508.0,480.0,738.0,,,,15500.0,12.0,2010.0,,,755000.0
sigma,421.6100093688479,42.300570993810425,,24.284751774483208,9981.26493237915,,,,,,,,,,,,,1.3829965467415934,1.1127993367127318,30.202904042525258,20.645406807709406,,,,,,181.0662065872166,,,,,,,,456.0980908409277,,161.31927280654173,441.86695529243417,438.7053244594708,,,,,386.58773804107426,436.5284358862568,48.62308143352019,525.4803834232025,0.5189106060898064,0.23875264627921178,0.5509158012954317,0.5028853810928913,0.8157780441442284,0.2203381983840307,,1.6253932905840505,,0.6446663863122295,,,24.689724768590235,,0.7473150101111093,213.80484145338042,,,,125.3387943517241,66.25602767664971,61.11914860172858,29.317330556781883,55.75741528187418,40.17730694453018,,,,496.12302445794415,2.7036262083595126,1.3280951205521074,,,79442.5028828866
zeros,0,0,,0,0,,,,,,,,,,,,,0,0,0,0,,,,,,861,,,,,,,,467,,1293,118,37,,,,,0,829,1434,0,856,1378,9,913,6,1,,0,,690,,,0,,81,81,,,,761,656,1252,1436,1344,1453,,,,1408,0,0,,,0
missing,0,0,0,259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,60.0,RL,65.0,8450.0,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,8.0,Typ,0.0,NA,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,NA,NA,NA,0.0,2.0,2008.0,WD,Normal,208500.0
1,2.0,20.0,RL,80.0,9600.0,Pave,NA,R

In [6]:
# split data in train and test
train, test = df.split_frame(ratios=[0.8], seed = 1)

# train = df

In [7]:
#automl
aml = H2OAutoML(max_models = 10000,
                #max_runtime_secs = 1800,  #if not set it's 1 hour
                balance_classes = True,
		seed = 1,
                sort_metric = "RMSE")

In [8]:
# start training
# my leader board frame could be ROC AUC, Precision, Recall, F1, Accuracy, Gains/Lift. 
# Otherwise the best cross validation model would be chose
aml.train(training_frame = train, y = 'SalePrice')

AutoML progress: |
09:00:32.447: AutoML: XGBoost is not available; skipping it.

███████████████
09:00:55.594: DeepLearning_1_AutoML_20220930_090032 [DeepLearning def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for DeepLearning model: DeepLearning_1_AutoML_20220930_090032_cv_1.  Details: ERRR on field: _balance_classes: balance_classes requires classification.


█████████████████████████████████████████| 100%


In [9]:
#evaluation
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20220930_090032,27702.2,7.67412e+08,7.67412e+08,16012.9,0.133891
GBM_4_AutoML_20220930_090032,27986.1,7.83221e+08,7.83221e+08,16586.9,0.137958
GBM_3_AutoML_20220930_090032,28068.4,7.87838e+08,7.87838e+08,16716,0.138686
StackedEnsemble_BestOfFamily_AutoML_20220930_090032,28114.6,7.9043e+08,7.9043e+08,16507,0.136614
GBM_grid__1_AutoML_20220930_090032_model_4,28174.7,7.93811e+08,7.93811e+08,16386.9,0.138403
GBM_grid__1_AutoML_20220930_090032_model_5,28609.8,8.18523e+08,8.18523e+08,16489.6,0.13795
GBM_2_AutoML_20220930_090032,28734.4,8.25667e+08,8.25667e+08,16867,0.138622
GBM_grid__1_AutoML_20220930_090032_model_3,28761.1,8.27202e+08,8.27202e+08,16565.9,0.137986
GBM_grid__1_AutoML_20220930_090032_model_8,28784.3,8.28536e+08,8.28536e+08,16529.5,0.140017
GBM_grid__1_AutoML_20220930_090032_model_7,28939.3,8.37483e+08,8.37483e+08,17049,0.143623


In [10]:
# best model, getting the name and the statistics
best_model = aml.leader
best_model_name = best_model.model_id
print(best_model)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20220930_090032

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 136196023.97545508
RMSE: 11670.3052220349
MAE: 6568.039171331919
RMSLE: 0.05845933529277153
R^2: 0.9772459257920392
Mean Residual Deviance: 136196023.97545508
Null degrees of freedom: 1164
Residual degrees of freedom: 1160
Null deviance: 6973184954977.986
Residual deviance: 158668367931.40515
AIC: 25138.1174912631

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 767411872.4077069
RMSE: 27702.1997756082
MAE: 16012.875413838407
RMSLE: 0.1338911166873735
R^2: 0.8717896001429377
Mean Residual Deviance: 767411872.4077069
Null degrees of freedom: 1164
Residual degrees of freedom: 1160
Null deviance: 6983730603085.642
Residual deviance: 894034831354.9785
AIC: 27152.319133065597



In [11]:
#saving the model
model_path = h2o.save_model(model = best_model, path = '../model', force = True)

In [12]:
#loading the model
model = h2o.load_model(path = '../model/' + best_model_name)


In [13]:
#predict and getting data out from h2o
predictions = h2o.as_list(model.predict(df_sample))

stackedensemble prediction progress: |████████████████████████████████████| 100%


/Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'MSZoning' has levels not trained on: [NA]
  warnings.warn(w)
/Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Utilities' has levels not trained on: [NA]
  warnings.warn(w)
/Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior1st' has levels not trained on: [NA]
  warnings.warn(w)
/Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior2nd' has levels not trained on: [NA]
  warnings.warn(w)
/Users/hectorcozar/Documents/GitHub/house_prizes_kaggle_python/venv/lib/python3.9/site-packages/h2o/job.py:70: UserWarning: T

In [14]:
#finishing h2o
h2o.shutdown()

/var/folders/yj/nrb4536x26s8lfxkz_6kzf100000gn/T/ipykernel_53456/1621550077.py:2: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


H2O session _sid_8fe5 closed.


In [15]:
#merging predictions with the full data set
submission = pd.merge(test_df['Id'],
                          predictions,
                          left_index = True,
                          right_index = True)
submission.rename(columns = {'predict': 'SalePrice'}, inplace = True)

In [16]:
# save to csv
submission.to_csv('../data/submission_' + best_model_name + '.csv',
                  header = True, index = False)

# 00. Score of 0.12868
# 01. Change from 25 max models to 1000. Score of 0.12792
# 02. Sorting leaderboard by rmse as the main metric on h2o training. Score of 0.12792 made the same.
# 02. Gonna make models for 30 min instead doing 1000 models. No improvement 0.13452
# 02. We're going with 10.000 models. Score of 0.12970... no improving
# 03. Trying with xgboost